https://medium.com/saarthi-ai/build-a-smart-question-answering-system-with-fine-tuned-bert-b586e4cfa5f5

https://huggingface.co/transformers/torchscript.html

In [1]:
import pandas as pd

In [3]:
qa = pd.read_csv("watashiwa.csv")

In [45]:
reponses = pd.read_excel("watashiwa.xlsx")

In [77]:
paras = []
para = ""
for a in reponses['answer']:
    if (len(para.split(" ")) + len(a.split(" "))) > 400:
        paras.append(para)
        para = ""
        para += " " + a
    else:
        para += " " + a

In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Load the BertForQuestionAnswering model and the tokenizer.

In [50]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Create a QA example and use function encode_plus() to encode the example. The function encode_plus() returns a dictionary that contains input_ids, token_type_ids, and attention mask but we only need input_ids and token_type_ids for the QA task.

In [83]:
def answerme(question):
    answers = []
    for para in paras:
        encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special=True)

        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)

        end_index = torch.argmax(end_scores)

        answer = ' '.join(tokens[start_index:end_index+1])
        corrected_answer = ''

        for word in answer.split():

            #If it's a subword token
            if word[0:2] == '##':
                corrected_answer += word[2:]
            else:
                corrected_answer += ' ' + word
        answers.append(answer)
    return answers

In [ ]:
answerme("what is your name")

Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


In [78]:
question = '''watashiwa des'''

paragraph = paras[0]
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special=True)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


In [62]:
question = '''Why was the student group called "the Methodists?"'''

paragraph = ''' The movement which would become The United Methodist Church began in the mid-18th century within the Church of England.
            A small group of students, including John Wesley, Charles Wesley and George Whitefield, met on the Oxford University campus.
            They focused on Bible study, methodical study of scripture and living a holy life.
            Other students mocked them, saying they were the "Holy Club" and "the Methodists", being methodical and exceptionally detailed in their Bible study, opinions and disciplined lifestyle.
            Eventually, the so-called Methodists started individual societies or classes for members of the Church of England who wanted to live a more religious life. '''
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special=True)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


Run the QA example through the loaded model.

In [79]:
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

Now we have start scores and end scores we can get both the start index and the end index and use both the indices for span prediction.

In [80]:
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])

Note: The model is likely to predict an end word that is before the start word. The correct way is to pick a span for which the total score (start score + end score) is maximum where end_index ≥ start_index.

Note: BERT uses wordpiece tokenization. Wordpiece split the tokens like “playing” to “play and ##ing”. It also covers a wider spectrum of Out-Of-Vocabulary (OOV) words.

We can recover any words that were broken down into subwords with a little bit more work.

In [81]:
corrected_answer = ''

for word in answer.split():
    
    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

In [82]:
corrected_answer

' [CLS]'